# Variational Auto Encoder
The logic for this has been taken from the original paper and Lecture 13 of CS231n

## Model

In [6]:
from keras.src.backend.jax.numpy import argmax
from scipy.special import dtype
from torch.utils.data import DataLoader


class VariationalAutoencoder(nn.Module):

    def __init__(self, input_dim, hidden_dim = 128*128, latent_dim = 32*32):
        super(VariationalAutoencoder, self).__init__()

        #Encoding
        self.img2hid = nn.Linear(input_dim, hidden_dim)
        self.hid2mu = nn.Linear(hidden_dim, latent_dim)
        self.hid2var = nn.Linear(hidden_dim, latent_dim)

        #Decoding
        self.z2hid = nn.Linear(latent_dim, hidden_dim)
        self.hid2img = nn.Linear(hidden_dim, input_dim)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):

        h = self.relu(self.img2hid(x))
        mu = self.hid2mu(h)
        sigma = self.hid2var(h)

        return mu, sigma

    def decode(self, z):

        h = self.relu(self.z2hid(z))

        return self.sigmoid(self.hid2img(h))

    def forward(self, x):

        mu, sigma = self.encode(x)
        epsilon = torch.randn_like(sigma)
        z_reparametarized = mu + epsilon * sigma
        reconstructed = self.decode(z_reparametarized)

        return reconstructed, mu, torch.log(sigma)

In [181]:
import torch
from torch import nn

class LSTM(nn.Module):

    def __init__(self):
        super(LSTM, self).__init__()

        self.lstm1 = nn.LSTM(input_size=5, hidden_size=256, num_layers=2, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=256, hidden_size=128, num_layers=2, batch_first=True)
        self.fc1 = nn.Linear(in_features=128, out_features=32)
        self.fc2 = nn.Linear(in_features=32, out_features=2)
        # self.fc3 = nn.Linear(in_features=16, out_features=8)
        # self.fc4 = nn.Linear(in_features=8, out_features=2)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):

        out, (h, c) = self.lstm1(x)
        _, (h, c) =  self.lstm2(out)
        lh = h[-1]

        out = self.fc1(lh).squeeze(-1)
        out = self.relu(out)

        out = self.fc2(out)
        # out = self.relu(out)
        #
        # out = self.fc3(out)
        # out = self.relu(out)
        #
        # out = self.fc4(out)

        return out

## Data Generation and Augmentation

First part is AI generated.\
Dataset is generated on ~1 min clips of CSGO2 recorded at 720p @60 fps with H.264 encoding it was not running on AV1 encoding for me.

In [192]:
import torch
import numpy as np
import cv2

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.to('cuda')
model.eval()

Using cache found in C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-6-23 Python-3.12.3 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [193]:
def extract_features_from_csgo_video(video_path, output_path, max_frames=36000):   #This whole part is AI generated although logic was mine
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file.")
        return

    features = []
    ret, prev_frame = cap.read()

    if not ret:
        print("Error reading first frame.")
        return

    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    prev_brightness = prev_gray.mean()

    frame_count = 0
    h, w = prev_gray.shape
    center_x, center_y = w // 2, h // 2


    while True:

        ret, frame = cap.read()
        if not ret or frame_count >= max_frames:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        brightness = gray.mean()

        # Detect firing event (simple brightness spike)
        is_firing = 1 if (brightness - prev_brightness) > 15 else 0
        prev_brightness = brightness

        # Optical flow for dx, dy (approximates mouse movement)
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray,None, 0.5, 3, 15, 3, 5, 1.2, 0)
        dx = flow[center_y, center_x, 0]
        dy = flow[center_y, center_x, 1]
        prev_gray = gray

        # Dummy x, y target position: set to 0 (no detection)
        # You can add YOLOv8 to track enemies in future
        with torch.no_grad():
            results = model(frame)
        detections = results.xyxy[0]

        h, w, _ = frame.shape
        cx, cy = w // 2, h // 2  # crosshair center

        closest_dist = float('inf')
        x_pos, y_pos = 0.0, 0.0

        for *xyxy, conf, cls in detections:
            if int(cls) != 0:  # class 0 is 'person' in COCO
                continue
            x1, y1, x2, y2 = map(int, xyxy)
            px, py = (x1 + x2) / 2, (y1 + y2) / 2  # enemy center

            dx, dy = px - cx, py - cy
            dist = (dx**2 + dy**2)**0.5

            if dist < closest_dist:
                closest_dist = dist
                x_pos = dx
                y_pos = dy


        features.append([x_pos, y_pos, dx, dy, is_firing])
        frame_count += 1

    cap.release()

    features = np.array(features)
    print(f"Extracted {features.shape[0]} frames with {features.shape[1]} features each.")

    # Save as PyTorch tensor
    tensor = torch.tensor(features, dtype=torch.float32)
    torch.save(tensor, output_path)
    print(f"Saved features to: {output_path}")


In [194]:
idx = 20
for i in range(10,17):

    src = "C:/Users/VIDIT/Desktop/24035066-CSOC-IG/CSGO_model/dataset/clip" + str(i) + ".mp4"
    destination = "C:/Users/VIDIT/Desktop/24035066-CSOC-IG/CSGO_model/dataset/features_v" + str(idx) + ".pt"
    extract_features_from_csgo_video(src, destination)

    idx += 1

# features_v12 onwards
# features_ai10 onwards

C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models

Extracted 1803 frames with 5 features each.
Saved features to: C:/Users/VIDIT/Desktop/24035066-CSOC-IG/CSGO_model/dataset/features_v20.pt


C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models

Extracted 1825 frames with 5 features each.
Saved features to: C:/Users/VIDIT/Desktop/24035066-CSOC-IG/CSGO_model/dataset/features_v21.pt


C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models

...

C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models

Extracted 1803 frames with 5 features each.
Saved features to: C:/Users/VIDIT/Desktop/24035066-CSOC-IG/CSGO_model/dataset/features_v25.pt


C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models

Extracted 1811 frames with 5 features each.
Saved features to: C:/Users/VIDIT/Desktop/24035066-CSOC-IG/CSGO_model/dataset/features_v26.pt


C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\VIDIT/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [220]:
import os

search_in = os.listdir("C:/Users/VIDIT/Desktop/24035066-CSOC-IG/CSGO_model/dataset/")

In [221]:
path = "C:/Users/VIDIT/Desktop/24035066-CSOC-IG/CSGO_model/dataset/"
dataset = []
dataset1 = []
count_ai =0
count =0
for file in search_in:
    if(file.startswith("features_ai")):
        count_ai += 1
        dataset.append(torch.load(path + file))

for file in search_in:
    if(file.startswith("features_v")):
        count += 1
        dataset1.append(torch.load(path + file))

print(count_ai, count)

24 23


In [222]:
path = "C:/Users/VIDIT/Desktop/24035066-CSOC-IG/CSGO_model/dataset/"
dataset_test = []
dataset_test1 = []

for file in search_in:
    if(file.startswith("features_test")):
        dataset_test.append(torch.load(path + file))

for file in search_in:
    if(file.startswith("features_test_ai")):
        dataset_test1.append(torch.load(path + file))

In [223]:
min_len = min([len(i) for i in dataset]) if len(dataset) else float('inf')
min_len1 = min([len(i) for i in dataset1])
min_len = min(min_len, min_len1)
print(min_len)

1785


In [225]:
min_len_test = min([len(i) for i in dataset_test])
min_len_test1 = min([len(i) for i in dataset_test1])
min_len_test = min(min_len_test, min_len_test1)
print(min_len_test)

1799


In [227]:
for i, j in enumerate(dataset):

    j = j[:min_len]
    if i == 0:
        dataset_tensor_x_ai = torch.tensor(j, dtype=torch.float32)
    else:
        dataset_tensor_x_ai = torch.cat([dataset_tensor_x_ai, torch.tensor(j, dtype=torch.float32)], dim=0)

for i, j in enumerate(dataset1):

    j = j[:min_len]
    if i == 0:
        dataset_tensor_x = torch.tensor(j, dtype=torch.float32)
    else:
        dataset_tensor_x = torch.cat([dataset_tensor_x, torch.tensor(j, dtype=torch.float32)], dim=0)


In [226]:
for i, j in enumerate(dataset_test):

    j = j[:min_len_test]
    if i == 0:
        dataset_tensor_test = torch.tensor(j, dtype=torch.float32)
    else:
        dataset_tensor_test = torch.cat([dataset_tensor_test, torch.tensor(j, dtype=torch.float32)], dim=0)

for i, j in enumerate(dataset_test1):

    j = j[:min_len_test]
    if i == 0:
        dataset_tensor_test1 = torch.tensor(j, dtype=torch.float32)
    else:
        dataset_tensor_test1 = torch.cat([dataset_tensor_test1, torch.tensor(j, dtype=torch.float32)], dim=0)

In [228]:
print(dataset_tensor_x_ai.max(), dataset_tensor_x_ai.shape)
print(dataset_tensor_x.max(), dataset_tensor_x.shape)
print(dataset_tensor_test.max(), dataset_tensor_test.shape)
print(dataset_tensor_test1.max(), dataset_tensor_test1.shape)

tensor(623.) torch.Size([42840, 5])
tensor(623.) torch.Size([41055, 5])
tensor(561.50000) torch.Size([5397, 5])
tensor(531.50000) torch.Size([1799, 5])


In [229]:
n, _ = dataset_tensor_x_ai.shape
dataset_tensor_y_ai = torch.ones(n//600, dtype=torch.float32)

m, _ = dataset_tensor_x.shape
dataset_tensor_y = torch.zeros(m//600, dtype=torch.float32)

o, _ = dataset_tensor_test.shape
dataset_tensor_y_test = torch.zeros(o//600, dtype=torch.float32)

p, _ = dataset_tensor_test1.shape
dataset_tensor_y_test1 = torch.ones(p//600, dtype=torch.float32)

In [261]:
x = torch.concat([dataset_tensor_x_ai, dataset_tensor_x], dim=0)
y = torch.concat([dataset_tensor_y_ai, dataset_tensor_y], dim=0)

x_test = torch.concat([dataset_tensor_test, dataset_tensor_test1], dim=0)
y_test = torch.concat([dataset_tensor_y_test, dataset_tensor_y_test1], dim=0)

print(x.shape, y.shape)
print(dataset_tensor_x.shape, dataset_tensor_y.shape) #41.9%
print(dataset_tensor_x_ai.shape, dataset_tensor_y_ai.shape) #58.1%

print(x_test.shape, y_test.shape)
print(dataset_tensor_test.shape, dataset_tensor_y_test.shape)
print(dataset_tensor_test1.shape, dataset_tensor_y_test1.shape)

torch.Size([83895, 5]) torch.Size([139])
torch.Size([41055, 5]) torch.Size([68])
torch.Size([42840, 5]) torch.Size([71])
torch.Size([7196, 5]) torch.Size([10])
torch.Size([5397, 5]) torch.Size([8])
torch.Size([1799, 5]) torch.Size([2])


## Training

In [237]:
model = LSTM()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 20

In [238]:
torch.cuda.is_available()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [241]:
# model.state_dict(torch.load('C:/Users/VIDIT/Desktop/24035066-CSOC-IG/CSGO_model/weights.pth'))

model = model.to(device)
model.train()

LSTM(
  (lstm1): LSTM(5, 256, num_layers=2, batch_first=True)
  (lstm2): LSTM(256, 128, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=128, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=2, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
)

In [262]:
from torch.utils.data import  Dataset

class SequenceDataset(Dataset):
    def __init__(self, x, y, seq_len=600):
        self.x = x
        self.y = y
        self.seq_len = seq_len
        self.num_samples = min(len(y), x.shape[0] // seq_len) #WHY TF X.SHAPE IS 0

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        start = idx * self.seq_len
        end = start + self.seq_len
        x_seq = self.x[start:end]
        y_label = self.y[idx]
        return x_seq, y_label


In [263]:
dataset = SequenceDataset(x, y, 600)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)

In [265]:
for epoch in range(epochs):

    epoch_loss = 0
    total = 0
    correct = 0

    model.train()
    for x, y in dataloader:
        x = x.to('cuda')
        y = y.long().to('cuda')

        logits = model(x)
        loss = criterion(logits, y)
        epoch_loss += loss.item()

        prob = torch.softmax(logits, dim=1)
        predictions = torch.argmax(logits, dim=1)
        correct += (predictions == y).sum().item()
        total += y.size(0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch{epoch+1}, Loss:{epoch_loss}, Accuracy:{correct*100/total:.6f}")

Epoch1, Loss:96.58498412370682, Accuracy:48.920863
Epoch2, Loss:96.51084432005882, Accuracy:59.712230
Epoch3, Loss:95.6848343461752, Accuracy:53.237410
Epoch4, Loss:71.34169824048877, Accuracy:75.539568
Epoch5, Loss:75.38433448970318, Accuracy:70.503597
Epoch6, Loss:95.07162579894066, Accuracy:53.237410
Epoch7, Loss:83.66943266801536, Accuracy:69.784173
Epoch8, Loss:68.12294560670853, Accuracy:79.136691
Epoch9, Loss:62.93463345617056, Accuracy:79.856115
Epoch10, Loss:64.37094664387405, Accuracy:81.294964
Epoch11, Loss:47.20876679010689, Accuracy:86.330935
Epoch12, Loss:33.77820889907889, Accuracy:89.928058
Epoch13, Loss:26.99670248152688, Accuracy:93.525180
Epoch14, Loss:17.605089232558385, Accuracy:93.525180
Epoch15, Loss:3.957744972780347, Accuracy:98.561151
Epoch16, Loss:6.207690918818116, Accuracy:98.561151
Epoch17, Loss:1.7832744022016414, Accuracy:99.280576
Epoch18, Loss:3.0092212932067923, Accuracy:99.280576
Epoch19, Loss:1.791075152810663, Accuracy:99.280576
Epoch20, Loss:0.546

In [190]:
torch.save(model.state_dict(), 'C:/Users/VIDIT/Desktop/24035066-CSOC-IG/CSGO_model/weights.pth')

## Evaluation

In [234]:
model = LSTM()
model = model.to('cuda')

model.load_state_dict(torch.load('C:/Users/VIDIT/Desktop/24035066-CSOC-IG/CSGO_model/weights.pth'))
model.eval()

LSTM(
  (lstm1): LSTM(5, 256, num_layers=2, batch_first=True)
  (lstm2): LSTM(256, 128, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=128, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=2, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
)

In [266]:
test_set = SequenceDataset(x_test, y_test, 600)
testloader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=True)

In [267]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for x, y in testloader:
        x = x.to('cuda')
        y = y.long().to('cuda')

        logits = model(x)
        prob = torch.softmax(logits, dim=1)
        predicted = torch.argmax(prob, dim=1)

        correct += (predicted == y).sum().item()
        total += y.size(0)
    print(f"Test Accuracy: {correct*100/total}")


Test Accuracy: 80.0
